LangChainMemory
``` 
이전 대화 내용을 기억해서 문맥을 유지하는 역할 LangChain 0.3X부터는 LCEL기반으로 체인을 구성, RunnableWithMessageHistory, ChatMessageHistory등의 컴포넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결
```

In [6]:
%pip install --quiet langchain langchain-openai python-dotenv langchain_redis 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
import os

# 현재 작업 디렉토리에 있는 .env 파일 경로 지정
env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(env_path, override=True)  # 덮어쓰기 옵션 추가

print("현재 작업 디렉토리:", os.getcwd())
print("불러온 OPENAI_API_KEY:", repr(os.getenv("OPENAI_API_KEY")))

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
# 메시지 추가
history.add_user_message("안녕하세요")
history.add_ai_message("안녕하세요! 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

human: 안녕하세요
ai: 안녕하세요! 무엇을 도와드릴까요?


In [4]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})])

In [7]:
# Radis 기반 채팅 기록 저장소
from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
session_id = "my_session_id"  # 세션 ID 설정
history = RedisChatMessageHistory(redis_url=REDIS_URL, session_id=session_id)
# 메시지 추가
history.add_user_message("안녕하세요")
history.add_ai_message("안녕하세요! 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

ConnectionError: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.

In [18]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗봇
# 핵심 : session_id를 키로 하는 메모리 저장소만들고 사용자의 대화는 키별로 저장한다.
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뛰어난 한국어 상담 챗봇입니다. 질문에 친절하고 자세히 답변해주세요."),
    MessagesPlaceholder(variable_name="history"),  # ⬅️ 반드시 먼저 위치해야 함
    ("human", "{input}")                           # ⬅️ 사용자의 현재 입력은 마지막
])
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

In [19]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [20]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InMemoryChatMessageHistory를 생성
# get_session_history를 구현

In [23]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    '''세션 ID에 해당하는 대화 기록 객체를 반환합니다.
    존재하지 않는 세션 ID가 들어오면 새로운 InMemoryChatMessageHistory 객체를 생성합니다.
    '''
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(
    runnable=chain,                          # ✅ runnable로 지정
    get_session_history=get_session_history,  # ✅ get_session_history로 지정
    input_messages_key="input",          # 입력 메시지 키
    history_messages_key="history"        # 출력 메시지 키
)

In [27]:
# 두개의 세션을 번갈아가면서 대화
sessions = ["session1", "session2"]
questions = [
    "안녕하세요, 나는 짱구 임.",
    "안녕하세요, 나는 철수수 임.",
    "오늘 점심 추천해 주세요.",
    "오늘 저녁은 뭐 먹을까요?"
]
for i, question in enumerate(questions):
    session_id = sessions[i % len(sessions)]
    result = chatbot.invoke({"input": question}, config={'configurable': {'session_id': session_id}})
    print(f"Session {session_id} - 질문: {question}")
    print(f"Session {session_id} - 답변: {result}")


16:43:37 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session session1 - 질문: 안녕하세요, 나는 짱구 임.
Session session1 - 답변: 안녕하세요, 짱구! 만나서 반가워요. 무엇을 도와드릴까요? 질문이나 이야기가 있으면 언제든지 말씀해주세요. 함께 즐거운 시간 보내요!
16:43:39 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session session2 - 질문: 안녕하세요, 나는 철수수 임.
Session session2 - 답변: 안녕하세요, 철수수님! 만나서 반가워요. 저에게 궁금한 것이 있거나 이야기를 나누고 싶은 내용이 있으면 언제든지 말씀해주세요. 함께 대화를 즐기며 도와드릴게요. 어떤 것이든 물어보세요!
16:43:43 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session session1 - 질문: 오늘 점심 추천해 주세요.
Session session1 - 답변: 짱구님, 안녕하세요! 오늘 점심 추천해 드릴게요. 만약 한식을 좋아하신다면 비빔밥이나 불고기, 김치찌개 등이 좋을 것 같아요. 중식을 좋아하신다면 짜장면이나 탕수육을 추천해드릴게요. 양식을 좋아하신다면 피자나 파스타, 샐러드 등이 좋을 것 같아요. 어떤 음식이 땡기시나요? 좋아하는 음식이나 원하는 식사 스타일을 말씀해주시면 더 자세한 추천을 해드릴 수 있어요. 함께 맛있는 식사를 즐기세요!
16:43:45 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session

In [29]:
result = chatbot.invoke({"input": "저는 누구라구요?"}, config={'configurable': {'session_id': "session1"}})
print(result)

16:44:32 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
죄송합니다, 저는 여러분이 짱구라는 이름을 사용하셨다는 것을 인지하고 있습니다. 하지만 여러분이 짱구라는 실제 신원에 대해서는 알 수 없습니다. 제가 여러분을 도와드릴 수 있는 다른 질문이나 이야기가 있으시면 말씀해주세요. 어떤 도움이 필요하신가요? 함께 대화를 이어가요!


요약 메모리 구현(대화내용 자동 요약)
```
긴 대화내용을 모두 프롬프트에 기록하는 것은 비 효율적 -> 프롬프트의 길이 제한에 걸릴 가능성이 있음
Conversation Summary Memory
0.3x 버전에서는 직접 요약용 체인을 만들어서 ChatMessageHistory에 적용
```
어떻게 요약?
```
- 일정길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심내용만 남김
- 요약결과를 메모리에 시스템 메세지 등으로 저장->메모리 절약
- 새로운 사용자 입력시 요약된 맥락 + 최근 및 메시지만 참고해서 llm 전달
```

In [30]:
# 요약용 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뛰어난 한국어 상담 챗봇입니다. 다음 대화 내용을 요약해주세요."),
    ("human", "{conversation}")
])
# 요약 체인
summary_chain = summary_prompt | llm | StrOutputParser()

In [33]:
# 전체대화 내용을 요약하고 마지막 사용자 질문-답변 쌍만 원본 유지
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
# 요약 대상 대화내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages

if len(message) > 2:
    original_dialog = '\n'.join([f'{msg.type.upper()}: {msg.content}' for msg in message[:-2]])
else:
    original_dialog = '\n'.join([f'{msg.type.upper()}: {msg.content}' for msg in message])

# llm으로 요약 생성
summary_text = summary_chain.invoke({"conversation": original_dialog})
print("== 요약내용 ==")
print(summary_text)
# 요약된 내용과 마지막 QA 쌍을 합쳐서 새로운 대화 기록 생성
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content=f'요약:{summary_text}'))
# 최근 대화의 마지막 QA쌍 복원
if len(message) >= 2:
    last_user_message = message[-2]
    last_ai_message = message[-2]
    if isinstance(last_user_message, HumanMessage):
        new_history.add_user_message(last_user_message.content)
    else:
        new_history.messages.append(last_ai_message)
#메모리 교체
store[session_id] = new_history

17:49:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
== 요약내용 ==
요약: 짱구님과의 대화에서는 날씨, 운동, 짜장면, 가족과의 시간, 그리고 영화에 대한 이야기가 나눠졌습니다. AI는 상대방의 이야기에 공감하고 함께 대화를 즐기며 소통하는 모습을 보여주었습니다.


In [34]:
len(store[session_id].messages)  # 새로운 대화 기록의 메시지 개수 확인

2

In [32]:
# user_d 세션에 대화내용을 기록 긴 대화 생성
long_queries = [
    "안녕하세요, 나는 짱구 임. 오늘은 날씨가 참 좋네요. 당신은 어때요?",
    "저는 오늘 아침에 일찍 일어나서 운동을 했어요. 당신은 운동 좋아하세요?",
    "점심으로는 짜장면을 먹었는데, 정말 맛있었어요. 당신은 점심으로 뭐 드셨나요?",
    "저녁에는 가족과 함께 시간을 보냈어요. 가족과의 시간은 언제나 소중하죠.",
    "내일은 친구들과 영화 보러 갈 예정이에요. 어떤 영화를 볼지 고민 중이에요."
]
session_id = "user_d"
for q in long_queries:
    result = chatbot.invoke({"input": q}, config={'configurable': {'session_id': session_id}})
    print(f"Session {session_id} - 질문: {q}")
    print(f"Session {session_id} - 답변: {result}")

print(f'요약전 user_d의 메모리 메세지 개수 : {len(get_session_history(session_id).messages)}')
# 요약할 대화 내용 가져오기
print(store[session_id])

17:26:05 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_d - 질문: 안녕하세요, 나는 짱구 임. 오늘은 날씨가 참 좋네요. 당신은 어때요?
Session user_d - 답변: 안녕하세요, 짱구님. 날씨가 좋다니 다행입니다! 제가 날씨를 느끼진 않지만, 여러분이 좋은 날씨를 즐기고 계시다니 기쁩니다. 혹시 더 궁금한 것이 있으시면 언제든지 물어보세요. 함께 대화를 나누는 것을 즐기겠습니다. 😊
17:26:07 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_d - 질문: 저는 오늘 아침에 일찍 일어나서 운동을 했어요. 당신은 운동 좋아하세요?
Session user_d - 답변: 운동을 즐기시는 짱구님, 멋지십니다! 운동은 건강에 좋고 기분을 좋게 만들어주는 좋은 습관이에요. 제가 운동을 즐기진 않지만, 많은 분들이 운동을 통해 건강을 유지하고 스트레스를 해소하는 데 도움을 받는다고 해요. 짱구님은 어떤 종류의 운동을 즐기시나요? 함께 이야기를 나누며 즐거운 시간 보내보아요. 😊
17:26:09 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Session user_d - 질문: 점심으로는 짜장면을 먹었는데, 정말 맛있었어요. 당신은 점심으로 뭐 드셨나요?
Session user_d - 답변: 짱구님, 짜장면 맛있게 드셨군요! 제가 먹는 것은 없지만, 짜장면은 많은 분들이 좋아하는 음식 중 하나에요. 짜장면의 풍부한 맛과 향기를 상상해보며 대화를 나누는 것도 즐거운 시간이 될 것 같아요. 짱구님, 다음에는 또 어떤 맛있는 음식을 드시고 싶으신가요? 함께 대